In [31]:
%load_ext autoreload
%autoreload 2

In [32]:
import numpy as np
import cv2 as cv
import glob
import csv
import pickle
 
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*10,3), np.float32)
objp[:,:2] = np.mgrid[0:10,0:6].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob('Chessboard3/*.tiff')

print('read')

count = 0
for fname in images:
    count+=1
    print(f'read {count}')
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
 
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (10,6), None)
     
     # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        print(count)
        
        #Draw and display the corners
        cv.drawChessboardCorners(img, (10,6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

img = cv.imread('Chessboard3/6_Chessboard.tiff')
h, w = img.shape[:2]

newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

cv.destroyAllWindows()

read
read 1
read 2


KeyboardInterrupt: 

In [33]:
roi

(589, 499, 1412, 1542)

In [ ]:
img = cv.imread('Chessboard3/10_Chessboard.tiff')
h, w = img.shape[:2]

dst = cv.undistort(img, mtx, dist, None, newcameramtx)

x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

In [ ]:
# Save variables
data = {
    'mtx': mtx,
    'dist': dist,
    'newcameramtx': newcameramtx,
    'roi': roi,
}

with open('calibration_data.pkl', 'wb') as f:
    pickle.dump(data, f)